In [1]:
import bs4
import requests
import pandas as pd

In [50]:
def get_billboard_hot100(save_to_csv:[bool]=False, fp:[str]=None):
    """visit website, scrape the hot 100 songs and artists and return it as df"""
    response = requests.get(url="https://www.billboard.com/charts/hot-100/index.php")
    response.raise_for_status()
    website = response.text                             # read out the html code as text
    soup = bs4.BeautifulSoup(website, "html.parser")    # make soup
    li_items = soup.select("li.lrv-u-width-100p")
    
    songs = []
    artists = []
    for _ in range(len(li_items)):
        if _ % 2 == 0:
            this_scraped = li_items[_].get_text().replace("\n","_")
            this_entry = []
            for c in this_scraped.split("_"):
                if c != "" and len(c) > 1:
                    this_entry.append(c)
                    if len(this_entry) == 2:
                        break
            songs.append(this_entry[0])
            artists.append(this_entry[1])
    if save_to_csv:
        if fp:
            try:
                pd.DataFrame({"song":songs, "artist":artists}).to_csv(fp, index=False, mode="w+", sep=";")
            except PermissionError:
                print("File already exists.")
        else:
            print("No filepath given.")
    return pd.DataFrame({"song":songs, "artist":artists})

In [66]:
def find_song_or_artists(user_entry:[str]=None, fp:[str]=None):
    song_artist = pd.read_csv(fp, sep=";")
    print(song_artist.loc[song_artist['artist'].str.lower() == user_entry.lower()])
    print(song_artist.loc[song_artist['song'].str.lower() == user_entry.lower()])

In [68]:
find_song_or_artists("stay", "../data/song_artists.csv")

Empty DataFrame
Columns: [song, artist]
Index: []
   song                         artist
3  Stay  The Kid LAROI & Justin Bieber


In [51]:
get_billboard_hot100(True, "../data/song_artists.csv")

,song,artist
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz..."
1,Easy On Me,Adele
2,Heat Waves,Glass Animals
3,Stay,The Kid LAROI & Justin Bieber
4,Super Gremlin,Kodak Black
...,...,...
95,No Love,Summer Walker & SZA
96,Megan's Piano,Megan Thee Stallion
97,Eazy,The Game & Kanye West
98,Fish Scale,YoungBoy Never Broke Again


In [ ]:
### Steps to get to the get_billboard_hot100 function ###

In [2]:
# response = requests.get(url="https://www.billboard.com/charts/hot-100/index.php")
# response.raise_for_status()
# website = response.text                             # read out the html code as text
# soup = bs4.BeautifulSoup(website, "html.parser")    # make soup

In [3]:
# li_items = soup.find_all("li", {"class": "lrv-u-width-100p"})
# li_items = soup.select(".lrv-u-width-100p")
# li_items = soup.select("li.lrv-u-width-100p")
# print(len(li_items))

200


double the items as expected, on exploring found out, that the uneven index list entries are just a repetition of the numbers.<br>
**→** on looping use only the even list entries.

explore one (even) entry (replace the excessive *\n* with underscore:

In [5]:
# x = li_items[6].get_text().replace("\n","_")
# x

'____Stay___The Kid LAROI & Justin Bieber__________4_____1_____30____________4_____1_____30______'

the first non _ entry is the songname, the second the artist's name.<br>
to create a df, make two lists for songs / artists, loop through the even 

In [6]:
# songs = []
# artists = []
# for _ in range(len(li_items)):
#     if _ % 2 == 0:
#         this_scraped = li_items[_].get_text().replace("\n","_")
#         this_entry = []
#         for c in this_scraped.split("_"):
#             if c != "" and len(c) > 1:
#                 this_entry.append(c)
#                 if len(this_entry) == 2:
#                     break
#         songs.append(this_entry[0])
#         artists.append(this_entry[1])

In [7]:
# working for 99 entries:
# ranks = []
# songs = []
# interprets = []
# for _ in range(len(li_items)):
#     if _ % 2 == 0:
#         a = li_items[_].get_text()
#         songs.append(a.strip("\n").split("\n")[0])
#         interprets.append(a.strip("\n").split("\n")[3])
#         ranks.append(int(_/2+1))

In [8]:
# top100 = pd.DataFrame({"song":songs,
#                        "artist":artists
#                       })
# top100.head()

,song,artist
0,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz..."
1,Easy On Me,Adele
2,Heat Waves,Glass Animals
3,Stay,The Kid LAROI & Justin Bieber
4,Super Gremlin,Kodak Black


In [9]:
# top100.to_csv("../data/song_artists.csv", index=False)